# Medical Image Segmentation using DIGITS

Medical professionals commonly use cardiac MRIs to help diagnose various heart conditions such as hypertrophy (enlargement and thickening of the left ventricle walls), heart failure with myocardial infarction (heart attack) and heart failure without myocardial infarction.  The process of reviewing MRIs is time consuming – time that either the patient or the doctor may not have.  One of the more common deep learning computer vision tasks – image segmentation – can be applied to cardiac MRIs to assist doctors in quickly and automatically locating the area of the left ventricle for faster diagnosis.

In this lab, we will be using NVIDIA DIGITS to architect, train and validate a neural network that learns to locate the left ventricle in cardiac MRIs.  We will be using the 2009 Cardiac MR Left Ventricle Segmentation Challenge dataset. The dataset consists of 16-bit MRI images in DICOM format and expert-drawn contours in text format (coordinates of contour polylines).  This dataset was originally published with the following paper:

Radau P, Lu Y, Connelly K, Paul G, Dick AJ, Wright GA. “Evaluation Framework for Algorithms Segmenting Short Axis Cardiac MRI.” The MIDAS Journal – Cardiac MR Left Ventricle Segmentation Challenge, http://hdl.handle.net/10380/3070

## Process
The approach we take in this lab is as follows:

•	Announce the Sunnybrook cardiac images dataset to DIGITS and have DIGITS create training and validation files automatically;
<br>
•	Use a custom network based on AlexNet architecture to train a model;
<br>
•	Validate the new model;
<br>
•	Modify the existing model by introducing the Dice Metric;
<br>
•	Utilize transfer learning to further improve results;
<br>
•	Create an additional model that performs finer grain image segmentation.




## Creating the dataset in DIGITS

[Click here](/digits/) to open DIGITS.  
<br>
<br>

Any time you need to return to the DIGITS home page simply click on DIGITS found in the upper left corner as shown in the screenshot below:
<br>
<br>

![digits](Digits.PNG)
<br>
<br>

We will be using a custom plug-in to load the data into DIGITS.
DIGITS plug-ins are thin layers that users can implement to load data that are not in a format that DIGITS natively supports.
Plug-ins define a list of options to present to the user as part of the data creation form.
Plug-ins also take care of loading the data in Python into a format that DIGITS understands: the ubiquitous `numpy.array` object.
See for example the code for the [Image Segmentation plug-in](https://github.com/NVIDIA/DIGITS/tree/278d24b108c5c195ca3d07da82490e187012a2d4/digits/extensions/data/imageSegmentation).

The `Sunnybrook` plug-in reads 16-bit images from DICOM files (those images are the input of the network we will train) and creates black-and-white images out of the .txt files that delineate the contours of the left ventricle (those black-end-white images will be our labels).

On the home page, click `Datasets > New Dataset > Images > Sunnybrook LV Segmentation`.
<br>
<br>


![create_dataset](Create_Dataset.PNG)
<br>
<br>

In the dataset creation form:
- set `image folder` to: **/data/challenge_training**
- set `contour folder` to: **/data/Sunnybrook Cardiac MR Database ContoursPart3**
<br>
<br>

![image_contour](Image_Contour.PNG)
<br>
<br>

- set `feature encoding` to `none`
- set `encoder batch size` to `1`
- set `dataset name` to `Sunnybrook`
- click `Create`.
<br>
<br>

![encoding_create](Encoding_Create.PNG)
<br>
<br>

When the dataset is created you can refresh the page to review the data shapes:
- features (MRI images) are 256x256 16-bit images
- labels (contour images) are 256x256 black-and-white images
<br>
<br>

![dataset_details](Dataset_Details.PNG)
<br>
<br>

A sample feature/label pair may look like:
<br>
<br>
![feature](img.png)
<br>
<br>
![label](label.png)
<br>
<br>


## Creating a model

We will use a model from the FCN family.
These models were first introduced in the following paper:

> [Fully Convolutional Models for Semantic Segmentation](https://arxiv.org/abs/1411.4038)
> Jonathan Long*, Evan Shelhamer*, Trevor Darrell
> CVPR 2015
> arXiv:1411.4038
<br>

The first model we will create is the simplest model from this family of models.
It is derived from the popular AlexNet model.

> NOTE: AlexNet is a classification network but with a few minor modifications it can be converted into a segmentation network.
> See the above paper for more details.
<br>
<br>

On the DIGITS home page (click DIGITS in the top left corner), click `New Model > Images > Sunnybrook LV Segmentation`.
<br>
<br>

![create_model](Create_Model.PNG)
<br>
<br>

In the model creation form:
- set the number of epochs to `5`
- set the snapshot interval to `5`
- select the `Sunnybrook` dataset
- set the base learning rate to `1e-4`
<br>
<br>

![epochs_lr](Epochs_LR.PNG)
<br>
<br>

- click the `custom network` tab then paste this [net description](/nKQrP6kVL9JX/edit/fcn-alexnet.protobuf)
<br>
<br>

![custom_network](Custom_Network.PNG)
<br>
<br>

- name your model `fcn_alexnet-sunnybrook`
- click `Create`
<br>
<br>

![model_create](Model_Create.PNG)
<br>
<br>


You should see that after a few epochs the network has learnt to predict the location of the left ventricle with over 98% accuracy.
This means that 98% of pixels are correctly predicted as belonging to either the left ventricle or the background.
This seems very good!
<br>
<br>

![accuracy_task1](Accuracy_Task1.PNG)
<br>
<br>

Now let us test the model on an image from the validation set.
Scroll down to the model page and select `Image Segmentation` in `Visualization Method`.
In `Test a record from validation set` select `SC-HF-NI-3`.
Check `Show visualizations and statistics`.
Click `Test`.
<br>
<br>

![trainedmodel_task1](TrainedModel_Task1.PNG)
<br>
<br>

A new page opens up.
There you can see how the image was segmented.
Can you spot the location of the left ventricle? 

Oh no! The segmented image is simply showing the original image! The network didn't find a single pixel of the left ventricle!
How is that possible? 

It turns out that in this dataset, less than 2% of pixels represent the left ventricle. The network has just lazily learnt to classify everything as background.

This is a typical "class imbalance" problem.
<br>
<br>

![trainedmodel_task1_result](Validation.PNG)
<br>
<br>


## Dice metric

We need a better metric to evaluate the performance of the model.
A popular metric for this kind of problems is the [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient).

Let us add this metric to our model.
By default, Caffe, the underlying Deep Learning framework we are using, does not support this metric.
However this is easily added through a custom Python layer.

DIGITS makes it easy to define and upload a custom layer. Save this [custom layer](layers.py) as "layers.py" and review the code. You'll upload it to DIGITS soon.

Now use the `Clone` button on the model that you just trained to create a new model with the same settings.
In the custom network description, scroll down to the end and un-comment the `dice` each line *below* the title "Dice Coefficient."
<br>
<br>

![uncomment_dice](Uncomment_Dice.PNG)
<br>
<br>


In the 'Python layers' menu, check `client-side file` then upload `layers.py`.
<br>
<br>

![python_layers](Python_Layers.PNG)
<br>
<br>

Name your model `fcn_alexnet-sunnybrook-with_dice`.
Now click `Create`.
<br>
<br>

![model_create_dice](Model_Create_Dice.PNG)
<br>
<br>

You will see that even though the accuracy is very high, the Dice coefficient remains at zero.
Our model did not learn anything useful but at least we now have a metric to figure that out.
<br>
<br>

![dice_result](Dice_Result.PNG)
<br>
<br>

## Transfer learning

Part of the reason why we are unable to learn anything is that our dataset is too small for our model.
We would need more samples.
An alternative is to do "transfer learning." I.E. Re-use knowledge that the model has acquired when training on another dataset and apply that knowledge to a different task.
Here we will use a pre-trained Alexnet model that was trained on the 1.2M ImageNet database (ILSVRC2012).

In order to be able to use the pre-trained Alexnet model, we need to make a modification to our dataset.  The Sunnybrook dataset is currently in grayscale; however, the pre-trained model is expecting three channels - RGB.  DIGITS provides an easy way to make this change.  

Go to your datasets page and clone the Sunnybrook dataset. Change Channel Conversion setting from Grayscale to RGB as shown below. 
<br>
<br>

![channel_conversion](Channel_Conversion.PNG)
<br>
<br>

Name the new copy Sunnybrook-RGB and click create.
<br>
<br>

Clone the last model you trained.
Select the new Sunnybrook-RGB dataset and set the number of epochs to `20`.

Remember to re-upload your Python layer.

Check the box `Show advanced learning rate options` and set the learning rate policy to `fixed`.
<br>
<br>

![advanced_lr](Advanced_LR.PNG)
<br>
<br>

In `pretrained model` point DIGITS to this path: `/data/fcn_alexnet.caffemodel`.
<br>
<br>

![pretrained_model](Pretrained_Model.PNG)
<br>
<br>

Name your model `fcn_alexnet-sunnybrook-with_dice-pretrained`.
Now click `Create`.  

NOTE: This will likely take around 10 minutes to complete.


After training you will see that the Dice coefficient exceeds 65%.
<br>
<br>
![pretrained_model2](Pretrained_Model2.PNG)
<br>
<br>

Now test the same image again.
<br>
<br>

![pretrained_results2](Pretrained_Results2.PNG)
<br>
<br>

You will notice a greyish / white blob on the image which corresponds to the network's prediction of the location of the left ventricle.
As you can see, the edges are pretty rough.
This is because the predictions in FCN-Alexnet are made at a 32-pixel stride, meaning that only one prediction is made for each block of 32x32 pixels.
This is too coarse.

## FCN-8s

We will now use a finer model called FCN-8s.  This model is able to make predictions at a much finer grain - down to 8x8 blocks.
Clone the model you just trained.  Remember to re-upload your custom Python layer.  Set the batch size to `1` to reduce the memory footprint during training and set the number of epochs to 3.  NOTE: Time constraints warrant the use of 3 epochs; however, in practice you would likely specify a higher epoch value.  The screen captures that follow show the results when the number of epochs is 20 (so that a comparison can be made with the AlexNet example from the previous step).

In the custom network tab paste the content of [`fcn-8s.protobuf`](/nKQrP6kVL9JX/edit/fcn-8s.protobuf).  In `pretrained model` point DIGITS to this path: `/data/fcn8s-heavy-pascal.caffemodel`.  Name your model `fcn_8s-sunnybrook-with_dice-pretrained`.


After training you should see that the model reaches a Dice coefficient of over 88%.
<br>
<br>

![fcn8_model](FCN8_Model.PNG)
<br>
<br>

Now test the same image again, you should see that the contour of the prediction is much smoother.
<br>
<br>

![fcn8_validation](FCN8_Validation.PNG)
<br>
<br>



## REFERENCE:  FCN-Alexnet and FCN-8s network descriptions

* [`fcn-alexnet.protobuf`](/nKQrP6kVL9JX/edit/fcn-alexnet.protobuf)
* [`fcn-8s.protobuf`](/nKQrP6kVL9JX/edit/fcn-8s.protobuf)
